In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

import requests
from bs4 import BeautifulSoup

from scrap_utils import Quote, convert_manual_author, extrat_quote, tokenize, get_babelio_citation, scrap_recent_babelio
%load_ext autoreload
%autoreload 2

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

def get_poem(soup):
    for ii, card in enumerate(soup.find_all('article', class_='entry-item')): # 'entry-description')):
        quote = card.p.text
        title = card.header.h1.text
        author = card.header.span.text[4:]
    return title, quote, author

url =  [ "https://www.poemes.co/paul-eluard.html",
        "https://www.poemes.co/jacques-prevert.html",
        "https://www.poemes.co/paul-verlaine.html",
        "https://www.poemes.co/victor-hugo.html",
        "https://www.poemes.co/charles-baudelaire.html",
        "https://www.poemes.co/alphonse-de-lamartine.html",
        "https://www.poemes.co/rainer-maria-rilke.html",
        "https://www.poemes.co/rene-francois-sully-prudhomme.html",
        "https://www.poemes.co/robert-desnos.html",
        "https://www.poemes.co/charles-leconte-de-lisle.html",
        "https://www.poemes.co/etienne-de-la-boetie.html",
        "https://www.poemes.co/guillaume-apollinaire.html",
        "https://www.poemes.co/rene-char.html",]

In [104]:
for url_author in url:
    print(url_author)
    list_lien_titre = []
    old_list_lien_titre = []
    for n_page in range(100):
        url = f"{url_author}?page={n_page}"
        response = requests.get(url, headers=headers)
        html_content = response.content # .decode('utf-8')
        soup = BeautifulSoup(html_content, 'html.parser')
        print(n_page, end = ".")
        new_list_lien_titre = []
        for ii, card in enumerate(soup.find_all('div', class_="widget kopa-article-list-widget categories-list-1")):
            for lien in card.find_all('ul'):
                new_list_lien_titre.append(lien.a.attrs['href'])

        if ".".join(new_list_lien_titre) == ".".join(old_list_lien_titre):
            break
        else:
            old_list_lien_titre = new_list_lien_titre
            list_lien_titre += new_list_lien_titre
            time.sleep(1)
            
    pd.DataFrame(list_lien_titre).to_csv(f"data_v2/scrap_poemesco/{url_author.split('/')[-1][:-4]}.csv", index = False)
    print("-")
    

https://www.poemes.co/jacques-prevert.html
0.1.2.3.4.5.6.7.8.-
https://www.poemes.co/paul-verlaine.html
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.-
https://www.poemes.co/victor-hugo.html
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.-
https://www.poemes.co/charles-baudelaire.html
0.1.2.3.4.5.6.7.-
https://www.poemes.co/alphonse-de-lamartine.html
0.1.2.3.4.-
https://www.poemes.co/rainer-maria-rilke.html
0.1.2.3.-
https://www.poemes.co/rene-francois-sully-prudhomme.html
0.1.2.3.4.5.6.7.-
https://www.poemes.co/robert-desnos.html
0.1.2.3.4.5.6.7.8.-
https://www.poemes.co/charles-leconte-de-lisle.html
0.1.2.3.4.5.6.7.-
https://www.poemes.co/etienne-de-la-boetie.html
0.1.2.-
https://www.poemes.co/guillaume-apollinaire.html
0.1.2.3.4.5.-
https://www.poemes.co/rene-char.html
0.1.2.3.4.5.6.-


In [4]:
l_author_ = os.listdir("data_v2/scrap_poemesco")

for author_ in l_author_:
    l_quote = []
    df = pd.read_csv(f"data_v2/scrap_poemesco/{author_}")
    list_titre = df[df.columns[0]]
    print(author_, len(list_titre))
    # if author_ in ["rene-char..csv", "paul-verlaine..csv"]:
    #     continue
    for ii, url_titre in enumerate(list_titre):
        print(ii, end = '.')
        url = "https://www.poemes.co/" + url_titre
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.get(url, headers=headers, timeout=300)
                break
            except requests.exceptions.ConnectTimeout:
                if attempt < max_retries - 1:
                    print(f"Timeout & Retry", end ='.')
                    time.sleep(35)
                else:
                    print(f"Timeout & Skip", end ='.')
                    continue
        html_content = response.content # .decode('utf-8')
        soup = BeautifulSoup(html_content, 'html.parser')
        try:
            title, quote, author = get_poem(soup)
            q = Quote(quote, author = author, title = title, source = "www.poemes.co", url = url)
            l_quote.append(q)
            time.sleep(10)
            pd.DataFrame([quote.__dict__ for quote in l_quote]).to_csv(f"data_v2/scrap_poemesco_poem/{author_}.csv", index = False)
        except Exception as e:
            print('-')
            print('--', ii, 'error:', url, e)
            
    pd.DataFrame([quote.__dict__ for quote in l_quote]).to_csv(f"data_v2/scrap_poemesco_poem/{author_}.csv", index = False)
    print("-")
    print("-")

rene-char..csv 170
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.-
-- 30 error: https://www.poemes.co//rene-char.html?page=1 local variable 'title' referenced before assignment
31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.-
-- 61 error: https://www.poemes.co//rene-char.html local variable 'title' referenced before assignment
62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.-
-- 92 error: https://www.poemes.co//rene-char.html local variable 'title' referenced before assignment
93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.-
-- 123 error: https://www.poemes.co//rene-char.html local variable 'title' referenced before assignment
124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.-
-- 154 error: https://www.poemes.co//rene

In [5]:
df = pd.read_csv('data_v2/scrap_poemesco_poem/alphonse-de-lamartine..csv.csv')
print(df['text'].iloc[0])

Lorsque seul avec toi, pensive et recueillie,  
      Tes deux mains dans la mienne, assis à tes côtés,  
      J'abandonne mon âme aux molles voluptés  
      Et je laisse couler les heures que j'oublie ;  
      Lorsqu'au fond des forêts je t'entraîne avec moi,  
      Lorsque tes doux soupirs charment seuls mon oreille,  
      Ou que, te répétant les serments de la veille,  
      Je te jure à mon tour de n'adorer que toi ;  
      Lorsqu'enfin, plus heureux, ton front charmant repose  
      Sur mon genou tremblant qui lui sert de soutien,  
      Et que mes doux regards sont suspendus au tien  
      Comme l'abeille avide aux feuilles de la rose ;  
      Souvent alors, souvent, dans le fond de mon coeur  
      Pénètre comme un trait une vague terreur ;  
      Tu me vois tressaillir; je pâlis, je frissonne,  
      Et troublé tout à coup dans le sein du bonheur,  
      Je sens couler des pleurs dont mon âme s'étonne.  
      Tu me presses soudain dans tes bras caressants,  
  

In [10]:
data_v1 = pd.read_parquet('data_v2/raw_data_v1_17_nov_.parquet') 
data_v1['note'] = data_v1['vo']
data_v1['all_search'] = (data_v1['text_tok'].astype(str)) + (data_v1['author'].astype(str).apply(tokenize)) + (data_v1['book'].astype(str).apply(tokenize)) + (data_v1['title'].astype(str).apply(tokenize)) + data_v1['quote_react'].astype(str)
data_v1.set_index('text_tok', inplace = True, drop = True)
data_v1.drop(['year', 'source', 'confiance', 'page', 'url', 'nb_char', 'nb_lines', 'sonnet', 'vo'], axis = 1, inplace = True)
data_v1['is_delete'] = False
data_v1.columns

Index(['text', 'author', 'book', 'title', 'nb_like', 'haiku', 'quote_react',
       'note', 'all_search', 'is_delete'],
      dtype='object')

In [14]:
l_author_ = os.listdir("data_v1.1/scrap_poemesco_poem")
l_df = []
for author in l_author_:
    l_df.append(pd.read_csv(f"data_v1.1/scrap_poemesco_poem/{author}"))
df = pd.concat(l_df)
df.drop(['year', 'source', 'confiance', 'page', 'url', 'nb_char', 'nb_lines', 'sonnet', 'vo'], axis = 1, inplace = True)
df['is_delete'] = False
df['note'] = None
df['all_search'] = (df['text'].astype(str)) + (df['author'].astype(str).apply(tokenize)) + (df['title'].astype(str).apply(tokenize))
df.set_index('text_tok', inplace = True, drop = True)
df['quote_react'] = None
df

,text,author,book,title,nb_like,haiku,is_delete,note,all_search,quote_react
text_tok,,,,,,,,,,
votre femme chantait delicieusement de tres anciens vers miens par vous mis en musique vers sans grande portee ideale ou physique mais que la voix etait exquise et l air charmant si bien que j entrais dans un grand etonnement moi le lasse qui reve d etre un ironique d ainsi revivre sensuel et platonique quoi sensuel vraiment platonique comment ah quand jeune j etais ainsi tiens tiens possible apres tout oui revasseur et mauvais sujet ma tete alors desirait et ma chair songeait mais j admire moi le blase mais l impassible non j admire combien la sympathie et l art evoquerent l enfant presque au quasi vieillard,Votre femme chantait délicieusement \n D...,Paul Verlaine,NaN,A Adrien Remacle,NaN,NaN,False,None,Votre femme chantait délicieusement \n D...,None
sonnet et nous voila tres doux a la betise humaine lui pardonnant vraiment et meme un peu touches de sa candeur extreme et des torts tres legers dans le fond qu elle assume et du train qu elle mene pauvres gens que les gens mourir pour celimene epouser angelique ou venir de nuit chez agnes et la briser et tous les sots peches tel est l amour encor plus faible que la haine l ambition l orgueil des tours dont vous tombez le vin qui vous imbibe et vous tord imbibes l argent le jeu le crime un tas de pauvres crimes c est pourquoi mon tres cher merat merat et moi nous etant depouilles de tout banal emoi vivons dans un dandysme epris des seules rimes extrait de recueil jadis et naguere,Sonnet. \n Et nous voilà très doux à la ...,Paul Verlaine,NaN,À Albert Mérat,NaN,NaN,False,None,Sonnet. \n Et nous voilà très doux à la ...,None
vous m avez pris dans un moment de calme familier ou le masque devient comme enfantin comme a nouveau tel j etais moins la barbe et ce front de tete de veau vers l an quarante huit bebe rotond en montpellier j allais dans des peyroux tranquillement avec ma bonne j y faisais mille et des fortins de sable inexpugnables et des fosses remplis mon dieu des eaux les moins potables suivant l exemple que gargantua pompier nous donne j y voyais passer des processions des penitents et proclamer la republique en ces candides temps ou tant d un tas d avis n etaient pas encore inventes mais malgre ce souci de nos jours qu il agite et trouble et d autres au trefonds de mes moelles encor butes je demeure assure conforme a votre excellent double,Vous m'avez pris dans un moment de calme famil...,Paul Verlaine,NaN,A Aman-Jean,NaN,NaN,False,None,Vous m'avez pris dans un moment de calme famil...,None
la grande sand porta sur les fonts baptismaux votre muse robuste et saine et bonne fee vous predit le genie et l oeuvre d un orphee charmant l homme et la femme et jusqu aux animaux jusqu au serpent jusqu a l oiseau sur les rameaux et vous pour faire bien la parole prouvee vous avez remporte ce double cher trophee belle ampleur de l idee en l aime ampleur des mots vos livres sont un don meme de la nature tant il fait bon les lire et les relire ainsi qu on respire et respire une atmosphere pure vos livres ou l amour qu il faut jamais transi toujours sincere eclate en vives splendeurs franches puis ou le male au fond qu on est prend ses revanches,La grande \n Sand porta sur les fonts ba...,Paul Verlaine,NaN,A Armand Silvestre,NaN,NaN,False,None,La grande \n Sand porta sur les fonts ba...,None
habitant de ces chers confins de la bastille ou je fus trop heureux et puis trop malheureux battant monnaie ici la faisant buisson creux et passant c est le mot de l amer a la fille tous accrocs et raccrocs dont mon dossier fourmille ami dans ces quartiers moi qui berce par eux berne par eux d amours bizarres et d affreux guignons leur garde comme un regret de famille je vous prie instamment du fond de ce broussais un hopital sis a plaisance ou le poete vit caresse par l ombre du drapeau francais de porter mon bonjour et mon baiser de fete a ce mien passe d or vanne represente par un genie en l air miser